In [0]:
!pip install socketIO_client_nexus

     |████████████████████████████████| 204kB 4.3MB/s 


# Realtime data from IEX 

In [0]:
from socketIO_client_nexus import SocketIO, BaseNamespace
import json
import zmq


_SIO_URL_PREFIX = 'https://ws-api.iextrading.com'
_SIO_PORT = 443


def _tryJson(data, raw=True):
    '''internal'''
    if raw:
        return data
    try:
        return json.loads(data)
    except ValueError:
        return data


class WSClient(object):
    def __init__(self, addr, tickers=None, on_data=None, on_open=None, on_close=None, raw=False, tcp='tcp://127.0.0.1:7000'):
        '''
           addr: path to sio
           sendinit: tuple to emit
           on_data, on_open, on_close: functions to call
       '''

        # Socket to talk to server
        # Global variable as need to pass to function 
        context = zmq.Context()
        zmqsocket = context.socket(zmq.PUB)
        zmqsocket.connect(tcp)
        print('IEX publisher connect to port 7000')
        
        # connect to correct socket 
        self.addr = addr
        self.tickers = tickers

        on_data = on_data or print 

        class Namespace(BaseNamespace):
            def on_connect(self, *data):
                if on_open:
                    on_open(_tryJson(data, raw))

            def on_disconnect(self, *data):
                if on_close:
                    on_close(_tryJson(data, raw))

            def on_message(self, data):
                prased = _tryJson(data, raw)
                on_data(prased)
                zmqsocket.send_multipart([bytes('IEX', 'utf-8'), bytes(json.dumps(prased), 'utf-8')])

        self._Namespace = Namespace

    def run(self):
        self.socketIO = SocketIO(_SIO_URL_PREFIX, _SIO_PORT)
        self.namespace = self.socketIO.define(self._Namespace, self.addr)
        for t in self.tickers:
            if self.addr == '/1.0/tops':
                self.namespace.emit('subscribe', t)
                print('Subscribe to IEX {}'.format(t))
            if self.addr == '/1.0/deep':
                subscribe_dict = {'channels':['deep']}
                subscribe_dict['symbols'] = [t]
                self.namespace.emit('subscribe', json.dumps(subscribe_dict))
                print('Subscribe to IEX DEEP {}'.format(t))
        self.socketIO.wait()



## TOPS
trades and the best bid and ask 

In [0]:
IEX_TOPS = WSClient('/1.0/tops', ['SPY','QQQ','BLK'])
IEX_TOPS.run()

IEX publisher connect to port 7000
Subscribe to IEX SPY
Subscribe to IEX QQQ
Subscribe to IEX BLK
{'symbol': 'SPY', 'sector': 'n/a', 'securityType': 'etp', 'bidPrice': 335.32, 'bidSize': 500, 'askPrice': 335.4, 'askSize': 500, 'lastUpdated': 1582216925012, 'lastSalePrice': 335.32, 'lastSaleSize': 100, 'lastSaleTime': 1582216923981, 'volume': 516536, 'marketPercent': 0.01927, 'seq': 219422}
{'symbol': 'QQQ', 'sector': 'n/a', 'securityType': 'etp', 'bidPrice': 233.39, 'bidSize': 400, 'askPrice': 233.56, 'askSize': 400, 'lastUpdated': 1582216924952, 'lastSalePrice': 233.445, 'lastSaleSize': 100, 'lastSaleTime': 1582216924123, 'volume': 168710, 'marketPercent': 0.0095, 'seq': 185077}
{'symbol': 'BLK', 'sector': 'diversifiedfinancials', 'securityType': 'commonstock', 'bidPrice': 0.0, 'bidSize': 0, 'askPrice': 0.0, 'askSize': 0, 'lastUpdated': 1582216892210, 'lastSalePrice': 562.09, 'lastSaleSize': 10, 'lastSaleTime': 1582216576213, 'volume': 8298, 'marketPercent': 0.04942, 'seq': 291}
{'sym

KeyboardInterrupt: ignored

##DEEP

Orderbook and other trade events 

In [0]:
IEX_DEEP = WSClient('/1.0/deep', ['SPY','QQQ','BLK'])
IEX_DEEP.run()

IEX publisher connect to port 7000
Subscribe to IEX DEEP SPY
Subscribe to IEX DEEP QQQ
Subscribe to IEX DEEP BLK
{'symbol': 'SPY', 'messageType': 'securityevent', 'data': {'securityEvent': 'MarketOpen', 'timestamp': 1582036200004}, 'seq': 1}
{'symbol': 'SPY', 'messageType': 'ophaltstatus', 'data': {'isHalted': False, 'timestamp': 1582029127946}, 'seq': 1}
{'symbol': 'SPY', 'messageType': 'tradingstatus', 'data': {'status': 'T', 'reason': '    ', 'timestamp': 1582029127946}, 'seq': 1}
{'symbol': 'SPY', 'messageType': 'ssr', 'data': {'isSSR': False, 'detail': ' ', 'timestamp': 1582029127946}, 'seq': 1}
{'symbol': 'SPY', 'messageType': 'book', 'data': {'bids': [{'price': 336.7, 'size': 2000, 'timestamp': 1582054597628}, {'price': 336.68, 'size': 200, 'timestamp': 1582054599721}, {'price': 336.67, 'size': 500, 'timestamp': 1582054564843}, {'price': 336.65, 'size': 600, 'timestamp': 1582054553733}, {'price': 336.62, 'size': 500, 'timestamp': 1582054502415}, {'price': 336.59, 'size': 100, 't

KeyboardInterrupt: ignored

## How do I get data for other asset classes?

### Currency ETFs

In [0]:
IEX_DEEP = WSClient('/1.0/deep', ['UUP','FXE','FXB'])
IEX_DEEP.run()

IEX publisher connect to port 7000
Subscribe to IEX DEEP UUP
Subscribe to IEX DEEP FXE
Subscribe to IEX DEEP FXB
{'symbol': 'UUP', 'messageType': 'securityevent', 'data': {'securityEvent': 'MarketOpen', 'timestamp': 1582036200004}, 'seq': 1}
{'symbol': 'UUP', 'messageType': 'ophaltstatus', 'data': {'isHalted': False, 'timestamp': 1582029127947}, 'seq': 1}
{'symbol': 'UUP', 'messageType': 'tradingstatus', 'data': {'status': 'T', 'reason': '    ', 'timestamp': 1582029127947}, 'seq': 1}
{'symbol': 'UUP', 'messageType': 'ssr', 'data': {'isSSR': False, 'detail': ' ', 'timestamp': 1582029127947}, 'seq': 1}
{'symbol': 'UUP', 'messageType': 'book', 'data': {'bids': [{'price': 26.85, 'size': 10000, 'timestamp': 1582054339439}, {'price': 26.84, 'size': 10000, 'timestamp': 1582048287828}, {'price': 26.83, 'size': 10000, 'timestamp': 1582047001757}, {'price': 26.82, 'size': 10000, 'timestamp': 1582052852786}], 'asks': [{'price': 26.86, 'size': 10000, 'timestamp': 1582045729120}, {'price': 26.87, '

KeyboardInterrupt: ignored

### Commodities ETFs

In [0]:
IEX_TOPS = WSClient('/1.0/tops', ['IAU','SLV'])
IEX_TOPS.run()

IEX publisher connect to port 7000
Subscribe to IEX IAU
Subscribe to IEX SLV
{'symbol': 'IAU', 'sector': 'n/a', 'securityType': 'etp', 'bidPrice': 15.29, 'bidSize': 275, 'askPrice': 15.3, 'askSize': 31500, 'lastUpdated': 1582054653373, 'lastSalePrice': 15.29, 'lastSaleSize': 100, 'lastSaleTime': 1582054172506, 'volume': 100824, 'marketPercent': 0.00602, 'seq': 17856}
{'symbol': 'SLV', 'sector': 'n/a', 'securityType': 'etp', 'bidPrice': 16.95, 'bidSize': 5500, 'askPrice': 16.96, 'askSize': 22700, 'lastUpdated': 1582054647230, 'lastSalePrice': 16.955, 'lastSaleSize': 100, 'lastSaleTime': 1582054580324, 'volume': 229381, 'marketPercent': 0.01495, 'seq': 21397}
{'symbol': 'IAU', 'sector': 'n/a', 'securityType': 'etp', 'bidPrice': 15.29, 'bidSize': 10275, 'askPrice': 15.3, 'askSize': 31500, 'lastUpdated': 1582054664573, 'lastSalePrice': 15.29, 'lastSaleSize': 100, 'lastSaleTime': 1582054172506, 'volume': 100824, 'marketPercent': 0.00602, 'seq': 17857}
{'symbol': 'SLV', 'sector': 'n/a', 'sec

KeyboardInterrupt: ignored

### Bond ETFs

In [0]:
IEX_TOPS = WSClient('/1.0/tops', ['SHY','TLH','TLT'])
IEX_TOPS.run()

IEX publisher connect to port 7000
Subscribe to IEX SHY
Subscribe to IEX TLH
Subscribe to IEX TLT
{'symbol': 'SHY', 'sector': 'n/a', 'securityType': 'etp', 'bidPrice': 84.95, 'bidSize': 52900, 'askPrice': 84.98, 'askSize': 62100, 'lastUpdated': 1582216942902, 'lastSalePrice': 84.965, 'lastSaleSize': 150, 'lastSaleTime': 1582216311916, 'volume': 1650, 'marketPercent': 0.00206, 'seq': 1977}
{'symbol': 'TLH', 'sector': 'n/a', 'securityType': 'etp', 'bidPrice': 151.94, 'bidSize': 1000, 'askPrice': 152.04, 'askSize': 1000, 'lastUpdated': 1582216943132, 'lastSalePrice': 151.73, 'lastSaleSize': 100, 'lastSaleTime': 1582213111639, 'volume': 100, 'marketPercent': 0.00434, 'seq': 12769}
{'symbol': 'TLT', 'sector': 'n/a', 'securityType': 'etp', 'bidPrice': 146.66, 'bidSize': 1300, 'askPrice': 146.72, 'askSize': 1300, 'lastUpdated': 1582216944772, 'lastSalePrice': 146.72, 'lastSaleSize': 138, 'lastSaleTime': 1582216927197, 'volume': 130266, 'marketPercent': 0.04054, 'seq': 58941}
{'symbol': 'TLT',

Question: What are the limitations of using the ETFs to estimate the price of the underlying?